In [5]:
filename = "Analysis_12_30_21_Colorado_Fire_segments.docx"

In [6]:
import sys
sys.path.append('../')

from helpers.utils import read_docx_to_dict

In [7]:
data = read_docx_to_dict(filename)

In [8]:
import pandas as pd
pd.options.display.max_rows = 500

# create dataframe
df = pd.DataFrame.from_dict(data)

In [9]:
df['text'] = df['text'].str.lower()

In [10]:
df.head()

,time,location,station,text
0,2021-12-30 6:14 PM,Salt Lake City,KTVX,u.s. many of these travel troubles will likely...
1,2021-12-30 6:12 PM,Phoenix (Prescott),KNXV,"500, 80 homes and businesses destroyed. it is ..."
2,2021-12-30 6:03 PM,Phoenix (Prescott),KNXV,reporting live from downtown flagstaff. luzdel...
3,2021-12-30 6:00 PM,San Francisco-Oak-San Jose,KGO,injured. the sheriff did not rule out the poss...
4,2021-12-30 5:54 PM,Tampa-St. Pete (Sarasota),WFTS,need this time. it's outside gambling groups a...


In [11]:
import sys
sys.path.append('../')

from helpers.utils import check_text_likeness

In [ ]:
df['matches'] = df.apply(lambda row: check_text_likeness(df, row['text']), axis=1)

In [ ]:
from helpers.utils import fetch_biggest_text, mark_use_row

In [ ]:
df['row_to_use'] = df.apply(lambda row: fetch_biggest_text(row['matches']), axis=1)

In [ ]:
mark_use_row(df)

In [ ]:
df.head(20)

In [ ]:
df['words'] = df['text'].str.lower().str.replace(',', '').str.replace('>', '').str.replace('.', '').str.replace('\n', '').str.replace('’', "'").str.replace(
    '!', '').str.replace('?', '').str.replace('%', '').str.replace(')', '').str.replace('(', '').str.replace('_', '').str.replace(':', '').str.strip().str.split(' ')

In [ ]:
df.head(20)

In [ ]:
import sys
sys.path.append('../')

In [ ]:
from helpers.utils import parse_words
df['clean_words'] = df.apply(lambda row: parse_words(row['words']), axis=1)

In [ ]:
df.head(20)

In [ ]:
from helpers.utils import fetch_climate_words_in_words, fetch_climate_phrases_in_text

In [ ]:
fetch_climate_words_in_words(["adapt","for", "climate", "change"])
# segment_df['climate_words_found'] = segment_df.apply(lambda row: fetch_climate_words_in_text(row['clean_words']), axis=1)

In [ ]:
fetch_climate_phrases_in_text("adapt for climate change")

In [ ]:
df['climate_words_found'] = df.apply(lambda row: fetch_climate_words_in_words(row['clean_words']), axis=1)

In [ ]:
df.head(20)

In [ ]:
df['climate_phrases_found'] = df.apply(lambda row: fetch_climate_phrases_in_text(row['text']), axis=1)

In [ ]:
df.head(20)

In [ ]:
# save data to csv
df.to_csv('reports/abc_all.csv', encoding='utf-8')

In [ ]:
unique_df = df[df['use_row']]

In [ ]:
unique_df

In [ ]:
total_words = unique_df['clean_words'].str.len().sum()
total_words

In [ ]:
def words_found_master_list(df_clean_words):
    """Given a column of words, aggregate master list"""
    words_found = list()
    for chunk in df_clean_words:
        words_found += chunk

    return words_found

In [ ]:
words_found = words_found_master_list(unique_df['clean_words'])
len(words_found)

In [ ]:
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import STOPWORDS

def master_stopwords_list():
    """Creates a master list of stopwords from pre-existing stopwords found in nltk and wordcloud"""
    stop_words = set(stopwords.words("english"))
    final_stopwords = list(STOPWORDS) + list(stop_words)
    return [i.lower() for i in set(final_stopwords)]

def lemmatize_words(words):
    """Given a list of words, distill to root words"""
    lem = WordNetLemmatizer()

    lemma_list = []
    for word, tag in pos_tag(words):
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma = word
        else:
            lemma = lem.lemmatize(word, pos=wntag)
        lemma_list.append(lemma)
    return lemma_list

def clean_lemmatized_words(lemma_words):
    """Removes stop words from the lemma list"""
    nonstop_lemma_words = []
    final_stopwords = master_stopwords_list()

    for word in lemma_words:
        if word not in final_stopwords:
            nonstop_lemma_words.append(word)

    return list(filter(None, nonstop_lemma_words))


In [ ]:
clean_lemma_words = clean_lemmatized_words(lemmatize_words(words_found))


In [ ]:
from nltk.probability import FreqDist

lfdist = FreqDist(clean_lemma_words)
lfdist

In [ ]:
import matplotlib.pyplot as plt
lfdist.plot(30,cumulative=False)
plt.show()

In [ ]:
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
from wordcloud import STOPWORDS
import matplotlib.pyplot as plt

wordcloud = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Set2', collocations=False, stopwords = master_stopwords_list()).generate_from_frequencies(lfdist)

# Plot
plt.figure( figsize=(15,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

#plt.savefig('word_cloud.png')

In [ ]:
import pandas as pd
pd.options.display.max_rows = 500
words_df = pd.DataFrame(lfdist.items(), columns=['Word', 'Count'])

words_df.sort_values(by=['Count'], ascending=False, inplace=True)
len(words_df)
# 1374 total words

words_df['Count'].sum()

# create data
climate_change_words_df = words_df.loc[words_df['Word'].isin(CLIMATE_CHANGE_RELATED_WORDS)]

climate_words_count = climate_change_words_df['Count'].sum()
non_climate_words_count = words_df['Count'].sum() - climate_words_count

comparison_df = pd.DataFrame({'Words': ['Climate-related', 'Non Climate-related'],
                             'counts': [climate_words_count, non_climate_words_count]})
comparison_df.set_index('Words', inplace=True)
print(comparison_df)

plot = comparison_df.plot.pie(y='counts', title="Climated-related vs non climated-related word frequencies", legend=True, autopct='%1.1f%%', shadow=True, figsize=(8, 8))

fig = plot.get_figure()
#fig.savefig("comparison.png")

In [ ]:
# find climate related word frequencies

# set figure size
fig, ax = plt.subplots(figsize=(12, 8))
# plot horizontal bar plot
climate_change_words_df.sort_values(by='Count').plot.barh(x="Word", y="Count", ax=ax)
# set the title
plt.title("Count of climate change related words")

for i, v in enumerate(climate_change_words_df['Count'].sort_values()):
    ax.text(v, i , str(v),
            color = 'blue', fontweight = 'bold')

plt.show()
# plt.savefig('climate-related-words-breakdown.png', transparent=False)

In [ ]:
# find segments
climate_change_words_found = list(climate_change_words_df['Word'].unique())
climate_change_words_found
    


In [ ]:
unique_df[unique_df["climate_words_found"].str.len() != 0].to_csv('reports/abc_final.csv', encoding='utf-8')